In [ ]:
import numpy as np
import healpy as hp
import emcee
from tqdm import tqdm
import matplotlib.pyplot as plt
import pyccl as ccl
import pandas as pd
from astropy.coordinates import SkyCoord
import os

Fluxo Completo Explicado:

    Leitura do Catálogo:

        Carrega o arquivo Pantheon+SH0ES.csv

        Carrega a matriz de covariância completa

        Adiciona índices às supernovas

        Filtra por intervalo de redshift

    Divisão em Pixels:

        Define a pixelização HEALPix (nside=4)

        Calcula as coordenadas do centro de cada pixel

        Cria diretório para armazenar os dados por pixel

    Associação de Supernovas:

        Para cada pixel, encontra todas as supernovas dentro de um raio de 90°

        Usa a fórmula de Haversine para calcular distâncias angulares

        Salva os dados de cada pixel em arquivos CSV separados

    Ajuste MCMC:

        Para cada pixel com dados suficientes (>5 supernovas):

            Extrai redshifts e magnitudes de distância

            Extrai a submatriz de covariância relevante

            Configura e executa o MCMC para estimar Ωₘ

            Armazena a mediana e o desvio padrão das amostras

    Resultados:

        Salva os mapas de Ωₘ e incertezas em arquivos FITS

        Gera visualizações em projeção Molweide

Observações Importantes:

    Controle de Qualidade: O código verifica se há dados suficientes em cada pixel antes de executar o MCMC.

    Paralelização: Para acelerar, você pode paralelizar o loop sobre os pixels usando multiprocessing.

    Visualização Adicional: Pode-se adicionar histogramas das distribuições de Ωₘ para pixels específicos.

    Adaptação: Os intervalos de redshift (z_min, z_max) e o raio do pixel podem ser ajustados conforme necessário.

In [ ]:
# ==============================================
# 1. LEITURA E PREPARAÇÃO DOS DADOS OBSERVACIONAIS
# ==============================================

# Parâmetros fixos
H0 = 67.27  # Valor fixo do Hubble
h = H0 / 100
Omega_b = 0.0494
sigma8 = 0.8120
n_s = 0.9646

# Carregar o catálogo Pantheon+
data = pd.read_csv('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES.csv', sep=' ')
cove = np.genfromtxt('/home/sofia/Documentos/Pantheon/Pantheon+SH0ES_STAT+SYS.cov', skip_header=1)
full_cov = np.reshape(cove, (1701, 1701))

# Adicionar índice às supernovas
data['index'] = np.arange(len(data))

# Filtrar por redshift (exemplo: 0.1 < z < 1.0)
z_min, z_max = 0.1, 1.0
data_filtered = data[(data['zCMB'] >= z_min) & (data['zCMB'] <= z_max)]

In [ ]:
# ==============================================
# 2. DIVISÃO EM PIXELS HEALPix (nside=4)
# ==============================================

nside = 4
npix = hp.nside2npix(nside)
raio = 90  # graus

# Obter coordenadas dos centros dos pixels
lon, lat = hp.pix2ang(nside, range(npix), lonlat=True)
galactic_coords = SkyCoord(l=lon, b=lat, frame='galactic', unit='deg')
equatorial_coords = galactic_coords.icrs
ra_pixel = equatorial_coords.ra.deg
dec_pixel = equatorial_coords.dec.deg

# Criar diretório para salvar os pixels
output_dir = '/home/sofia/Documentos/Pantheon/Atividade_4/Hemisferios_MCMC'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# ==============================================
# 3. ASSOCIAR SUPERNOVAS A CADA PIXEL
# ==============================================

# Converter coordenadas das supernovas para radianos
ra_data = np.radians(data_filtered['RA'].values)
dec_data = np.radians(data_filtered['DEC'].values)

# Para cada pixel, encontrar supernovas dentro do raio
pixel_data_list = []
for i in tqdm(range(npix), desc="Processando pixels"):
    # Coordenadas do centro do pixel
    ra_centro = np.radians(ra_pixel[i])
    dec_centro = np.radians(dec_pixel[i])
    
    # Fórmula de Haversine
    delta_ra = ra_data - ra_centro
    delta_dec = dec_data - dec_centro
    a = np.sin(delta_dec/2)**2 + np.cos(dec_centro)*np.cos(dec_data)*np.sin(delta_ra/2)**2
    distancia = np.degrees(2 * np.arcsin(np.sqrt(a)))
    
    # Selecionar supernovas dentro do raio
    mask = distancia <= raio
    pixel_data = data_filtered[mask].copy()
    
    # Salvar dados do pixel
    pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
    pixel_data.to_csv(pixel_file, index=False)
    pixel_data_list.append(pixel_data)

In [ ]:
# ==============================================
# 4. FUNÇÕES PARA O AJUSTE MCMC
# ==============================================

def mu_theory(z, Omega_m):
    """Calcula a magnitude de distância teórica usando PyCCL"""
    cosmo = ccl.Cosmology(
        Omega_c=Omega_m - Omega_b,
        Omega_b=Omega_b,
        h=h,
        sigma8=sigma8,
        n_s=n_s)
    a = 1/(1+z)
    return ccl.background.distance_modulus(cosmo, a)

def log_probability(theta, z_obs, mu_obs, inv_cov):
    """Função de log-probabilidade para o MCMC"""
    Omega_m = theta[0]
    
    # Prior uniforme
    if not 0.1 < Omega_m < 0.5:
        return -np.inf
    
    # Calcular predição teórica
    mu_theo = mu_theory(z_obs, Omega_m)
    
    # Calcular chi²
    delta = mu_obs - mu_theo
    chi2 = np.dot(delta, np.dot(inv_cov, delta))
    
    return -0.5 * chi2

In [ ]:
# ==============================================
# 5. EXECUÇÃO DO MCMC PARA CADA PIXEL
# ==============================================

# Configuração do MCMC

#nwalkers = 16
nwalkers = 4
#nsteps = 2000
nsteps = 80
burnin = 500

# Arrays para armazenar resultados
Om_map_mcmc = np.full(npix, np.nan)
Om_error_map = np.full(npix, np.nan)

In [ ]:
for i in tqdm(range(npix), desc="Ajustando pixels com MCMC"):
    pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
    pixel_data = pd.read_csv(pixel_file)
    
    # Verificar se há dados suficientes no pixel
    if len(pixel_data) < 5:  # Mínimo de 5 supernovas
        continue
    
    # Extrair dados observacionais
    z_obs = pixel_data['zCMB'].values
    mu_obs = pixel_data['MU_SH0ES'].values
    indices = pixel_data['index'].values
    
    # Extrair submatriz de covariância
    cov_sub = full_cov[np.ix_(indices, indices)]
    inv_cov_sub = np.linalg.pinv(cov_sub)
    if np.any(np.isnan(inv_cov_sub)) or np.any(np.isinf(inv_cov_sub)):
        print(f"Pixel {i}: matriz de covariância inválida!")
        continue
    
    # Configurar MCMC
    initial_guess = 0.3
    pos = initial_guess + 0.01 * np.random.randn(nwalkers, 1)
    
    sampler = emcee.EnsembleSampler(
        nwalkers, 1, log_probability,
        args=(z_obs, mu_obs, inv_cov_sub))
    
    # Executar MCMC
    sampler.run_mcmc(pos, nsteps, progress=False)
    
    # Processar resultados
    samples = sampler.get_chain(discard=burnin, flat=True)
    Om_map_mcmc[i] = np.median(samples)
    Om_error_map[i] = np.std(samples)

In [ ]:
i = 150  # Teste com o primeiro pixel
pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
pixel_data = pd.read_csv(pixel_file)

print("Número de supernovas:", len(pixel_data))
print("Redshifts:", pixel_data['zCMB'].values)
print("Magnitudes:", pixel_data['MU_SH0ES'].values)

In [ ]:
#Celula acima mas com log para debug

for i in tqdm(range(npix), desc="Ajustando pixels com MCMC"):
    try:
        print(f"\nProcessando pixel {i}...")  # Log adicional
        pixel_file = os.path.join(output_dir, f'pixel_{i:03d}.csv')
        pixel_data = pd.read_csv(pixel_file)
        print(f"Pixel {i}: {len(pixel_data)} supernovas")  # Log
        
        if len(pixel_data) < 5:
            print(f"Pixel {i} ignorado (poucos dados)")
            continue
            
        z_obs = pixel_data['zCMB'].values
        mu_obs = pixel_data['MU_SH0ES'].values
        indices = pixel_data['index'].values
        print(f"Pixel {i}: dados extraídos")  # Log
        
        cov_sub = full_cov[np.ix_(indices, indices)]
        print(f"Pixel {i}: submatriz extraída")  # Log
        
        inv_cov_sub = np.linalg.pinv(cov_sub)  # Usando pseudo-inversa
        print(f"Pixel {i}: matriz invertida")  # Log
        
        initial_guess = 0.3
        pos = initial_guess + 0.01 * np.random.randn(nwalkers, 1)
        
        sampler = emcee.EnsembleSampler(
            nwalkers, 1, log_probability,
            args=(z_obs, mu_obs, inv_cov_sub))
        
        print(f"Pixel {i}: iniciando MCMC...")  # Log
        sampler.run_mcmc(pos, nsteps, progress=True)  # Progresso ativado
        
        samples = sampler.get_chain(discard=burnin, flat=True)
        Om_map_mcmc[i] = np.median(samples)
        Om_error_map[i] = np.std(samples)
        print(f"Pixel {i} concluído")  # Log
        
    except Exception as e:
        print(f"ERRO no pixel {i}: {str(e)}")
        continue

In [ ]:
# ==============================================
# 6. SALVAR E VISUALIZAR RESULTADOS
# ==============================================

# Salvar mapas
# hp.write_map(os.path.join(output_dir, 'Om_map_mcmc.fits'), Om_map_mcmc, overwrite=True)
# hp.write_map(os.path.join(output_dir, 'Om_error_map.fits'), Om_error_map, overwrite=True)

valid_pixels = ~np.isnan(Om_map_mcmc)
hp.write_map(os.path.join(output_dir, 'Om_map_mcmc.fits'), Om_map_mcmc[valid_pixels], overwrite=True)
hp.write_map(os.path.join(output_dir, 'Om_error_map.fits'), Om_error_map[valid_pixels], overwrite=True)

# Visualização
projview(
    Om_map_mcmc,
    coord=["G"],
    unit="$\Omega_m$",
    title=f"Mapa de $\Omega_m$ (MCMC, ${z_min}<z<{z_max}$)",
    min=0.2, max=0.4,
    cmap='jet',
    graticule=True)
plt.savefig(os.path.join(output_dir, 'Om_map_mcmc.png'), dpi=300, bbox_inches='tight')
plt.close()

projview(
    Om_error_map,
    coord=["G"],
    unit="$\sigma(\Omega_m)$",
    title=f"Mapa de incertezas de $\Omega_m$",
    cmap='viridis',
    graticule=True)
plt.savefig(os.path.join(output_dir, 'Om_error_map.png'), dpi=300, bbox_inches='tight')
plt.close()

Explicação Detalhada do Método MCMC (Monte Carlo via Cadeias de Markov)

O MCMC (Monte Carlo via Cadeias de Markov) é um método computacional poderoso para estimar parâmetros e suas distribuições em problemas complexos de inferência estatística. Vou explicar passo a passo como ele funciona, usando como exemplo o seu caso de ajuste de Ωₘ em cosmologia.
1. Conceitos Fundamentais
a) O Problema que o MCMC Resolve

No seu caso, você quer encontrar a distribuição de probabilidade de Ωₘ dado seus dados observacionais (magnitudes de distância de supernovas). Matematicamente, queremos estimar:

P(Ωₘ | Dados) ∝ P(Dados | Ωₘ) × P(Ωₘ)

Onde:

    P(Ωₘ | Dados) é a distribuição posterior (o que queremos)

    P(Dados | Ωₘ) é a verossimilhança (que você calcula com PyCCL)

    P(Ωₘ) é a distribuição prévia (seus conhecimentos iniciais)

b) Por que MCMC?

Para problemas complexos:

    Não podemos calcular analiticamente a posterior

    Espaço de parâmetros é multidimensional

    Integrais são intratáveis analiticamente

2. Componentes do MCMC
a) Cadeia de Markov

Uma sequência de valores onde cada valor depende apenas do anterior (propriedade de Markov). No seu caso, cada "estado" da cadeia é um valor de Ωₘ.
b) Algoritmo de Metropolis-Hastings

O coração do MCMC usado no seu código (implementado pelo emcee):

    Estado Atual: Começa com um valor inicial de Ωₘ (ex: 0.3)

    Proposta: Gera um novo valor candidato baseado no atual (ex: 0.3 + ε)

    Avaliação: Calcula a razão entre as probabilidades posterior do novo e do atual valor

    Decisão: Aceita ou rejeita o novo valor com certa probabilidade

3. Funcionamento Passo a Passo no Seu Código
a) Configuração Inicial
python

nwalkers = 16      # Número de cadeias paralelas
nsteps = 2000      # Número de iterações
burnin = 500       # Período de "aquecimento"

b) Função de Verossimilhança
python

def log_probability(theta, z_obs, mu_obs, inv_cov):
    Omega_m = theta[0]
    
    # Prior uniforme (só aceita valores físicos)
    if not 0.1 < Omega_m < 0.5:
        return -np.inf
    
    # Calcula predição teórica com PyCCL
    mu_theo = mu_theory(z_obs, Omega_m)
    
    # Calcula chi² (considerando covariância)
    delta = mu_obs - mu_theo
    chi2 = np.dot(delta, np.dot(inv_cov, delta))
    
    return -0.5 * chi2  # Log da verossimilhança

c) Dinâmica do MCMC

    Inicialização:
    python

initial_guess = 0.3
pos = initial_guess + 0.01 * np.random.randn(nwalkers, 1)

Execução:
python

sampler.run_mcmc(pos, nsteps, progress=True)

Processamento:
python

    samples = sampler.get_chain(discard=burnin, flat=True)
    Om_map_mcmc[i] = np.median(samples)
    Om_error_map[i] = np.std(samples)

4. Visualização do Processo
a) Cadeias de Markov

Cada "walker" (caminhante) explora o espaço de parâmetros:
text

Walker 1: 0.300 → 0.305 → 0.302 → 0.308 → ...
Walker 2: 0.295 → 0.293 → 0.290 → 0.292 → ...
...
Walker 16: 0.310 → 0.308 → 0.312 → 0.315 → ...

b) Distribuição Estacionária

Após o burn-in, as cadeias convergem para a distribuição posterior:

https://i.imgur.com/XYZtrajectory.png
5. Por que Funciona para Seu Problema?

    Eficiência Computacional:

        Avalia PyCCL apenas em pontos relevantes

        Não precisa calcular toda a distribuição analiticamente

    Tratamento de Covariâncias:

        Incorpora naturalmente a matriz de covariância dos dados

    Estimativa de Incertezas:

        Obtém não só o valor médio mas toda a distribuição

6. Diagnósticos de Convergência

Para verificar se o MCMC funcionou bem:
a) Traço das Cadeias
python

plt.plot(sampler.get_chain()[:, :, 0].T, alpha=0.4)

As cadeias devem se misturar e não mostrar tendências.
b) Autocorrelação
python

autocorr_time = sampler.get_autocorr_time()

Tempos baixos indicam boa mistura.
c) Histograma dos Samples
python

plt.hist(samples, bins=30, density=True)

Deve mostrar uma distribuição suave.
7. Comparação com Seu Método Anterior
Característica	Ajuste por Minimização	MCMC
Estimativa de Ωₘ	Ponto único	Distribuição completa
Incertezas	Aproximada (Hessiana)	Direta dos samples
Tratamento de priors	Limitado	Incorporado naturalmente
Complexidade	Mais simples	Mais robusto
Custo computacional	Menor	Maior
8. Exemplo Prático no Seu Código

Quando você faz:
python

sampler.run_mcmc(pos, nsteps, progress=True)
samples = sampler.get_chain(discard=burnin, flat=True)

Está:

    Rodando 16 cadeias paralelas por 2000 passos cada

    Descartando as primeiras 500 iterações (burn-in)

    Obtendo ~24,000 amostras da distribuição posterior (16 walkers × 1500 passos úteis)

9. Visualização dos Resultados

A saída final são:

    Mapa de Ωₘ: Mediana da distribuição em cada pixel

    Mapa de incertezas: Desvio padrão das amostras

python

hp.mollview(Om_map_mcmc, title="Ωₘ MCMC")
hp.mollview(Om_error_map, title="Incertezas Ωₘ")

10. Dicas Práticas para Seu Caso

    Número de Walkers:

        16 é bom para 1 parâmetro (Ωₘ)

        Aumente para problemas multidimensionais

    Passos de Burn-in:

        Verifique visualmente quando as cadeias estabilizam

        500 é conservador para seu caso

    Priors:

        Você usou prior uniforme entre 0.1 e 0.5

        Pode usar priors informativas se tiver conhecimento prévio

    Diagnóstico:

        Sempre plote algumas cadeias para verificar convergência

        Verifique a autocorrelação

O MCMC é particularmente adequado para seu problema porque:

    O cálculo com PyCCL é computacionalmente intensivo (MCMC evita avaliações desnecessárias)

    Você quer estimativas robustas de incertezas

    O espaço de parâmetros é bem comportado (único mínimo)